In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict
import cv2 as cv

## Random Maps

In [ ]:
def load_map(path):
    with open(path, "r") as f:
        lines = f.readlines()
    height = int(lines[1].split()[1])
    width = int(lines[2].split()[1])
    cells = np.zeros((height, width))
    for i, line in enumerate(lines[4:]):
        for j, char in enumerate(line.strip()):
            if char == '@':
                cells[i, j] = 1
            elif char == '.' or char == "T":
                cells[i, j] = 0
            else:
                ValueError('Incorrect cell type')
    return cells

In [ ]:
load_dir = Path("/archive/savkin/raw_datasets/MIPT/TransPath_data/maps")

map = load_map(load_dir / "random-map" / "random512-40-9.map")

plt.imshow(map)
plt.show()

In [ ]:
random_maps_dir = load_dir / "random-map"
map_densities = np.arange(10, 41, 5)
random_maps = []
for density in tqdm(map_densities):
    for i in range(10):
        map = load_map(random_maps_dir / f"random512-{density}-{i}.map")
        random_maps.append(map)

random_maps = np.array(random_maps)
random_maps.shape

In [ ]:
np.save(random_maps_dir / "all_rand_maps.npy", random_maps)

## Street Maps

In [ ]:
street_maps_dir = load_dir / "street-map"
street_maps = defaultdict(list)
for path in tqdm(street_maps_dir.iterdir()):
    map = load_map(path)
    edge_size = map.shape[0]
    street_maps[edge_size].append(map)

In [ ]:
for edge_size, maps in street_maps.items():
    np.save(load_dir / "my_maps" / f"street_maps_{edge_size}.npy", maps)

## Tile MP Dataset

In [ ]:
path = "/home/logiczmaksimka/Documents/MIPT/10sem/heuristics/TransPath-Project/data/all_128x2.npz"

data = np.load(path)

Let's take a look at generated cf

In [ ]:

cf_pred = np.load("/home/logiczmaksimka/Downloads/val/cf_pred.npy")
cf_true = np.load("/home/logiczmaksimka/Downloads/val/cf.npy")

i = 0

p = cf_pred[0][0]
t = cf_true[0][0]


plt.imshow(p, vmin=0, vmax=1)
plt.show()

plt.imshow(t)
plt.show()

## Augmentations

In [ ]:
maps = np.load(load_dir / "my_maps" / f"street_maps_1024.npy")
maps.shape

In [ ]:
from scipy import ndimage

map = maps[0]

augmented_maps = []
for map in maps[:5]:
    rotation_types = [cv.ROTATE_90_CLOCKWISE, cv.ROTATE_180, cv.ROTATE_90_COUNTERCLOCKWISE]
    fliped_map = cv.flip(map, 0)
    augmented_maps = augmented_maps + [
        map,
        *[cv.rotate(map, rotation_type) for rotation_type in rotation_types],
        fliped_map,
        *[cv.rotate(fliped_map, rotation_type) for rotation_type in rotation_types]
    ]

## Generating start and goal nodes

Calc map hardness

In [ ]:
## обязательно проверь сложность карт!!!!!! Желательно прям добавить её к каждой карте!!!! Раз делаешь BFS то это совсем просто!!!